In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import lib_data_load as ld
import lib_data_ref as dr
from datetime import date
import seaborn as sns

start_year = 2020
end_year = 2020

dic_usagers = ld.load_usagers(start_year, end_year)
dic_vehic = ld.load_vehicules(start_year, end_year)

df_usagers = dic_usagers[2020]
df_vehic = dic_vehic[2020]

In [3]:
print(df_vehic.columns)

Index(['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm',
       'choc', 'manv', 'motor', 'occutc'],
      dtype='object')


## Merge data et suppression des colonnes inutiles

In [5]:
#  merge usagers and caracteristiques
df = df_usagers.merge(right=df_caract, on='Num_Acc', how='left')

# inutiles : ['id_vehicule', 'num_veh', 'adr'] 
df = df.drop(columns=['id_vehicule', 'num_veh', 'adr', 'lat', 'long'], axis=1)

NameError: name 'df_caract' is not defined

## Encodages

In [ ]:
# création de 4 classes basée sur les quartiles
# df['age'].describe()
def get_cl_age(age):
    if age <= 25:
        return '0-25'
    if 25 < age <= 37:
        return '26-37'
    if 37 < age <= 53:
        return '38-53'
    if 53 < age:
        return '>53'

df['luminosité'] = df['lum'].apply(lambda i: dr.get_labels('lum', i))
df['gravité'] = df['grav'].apply(lambda i: dr.get_labels('grav', i))
df['age'] = 2022 - df['an_nais']
df["classe d'age"] = [get_cl_age(age) for age in df['age']]
df["catégorie d'utilisateur"] = df['catu'].apply(lambda i: dr.get_labels('catu', i))
df["sexe_lbl"] = df['sexe'].apply(lambda i: dr.get_labels('sexe', i))
df['date'] = df[['jour', 'mois', 'an']].apply(lambda row : date(row['an'], row['mois'], row['jour']), axis=1)
df["agg_lbl"] = df['agg'].apply(lambda i: dr.get_labels('agg', i))
df["int_lbl"] = df['int'].apply(lambda i: dr.get_labels('int', i))
df["atm_lbl"] = df['atm'].apply(lambda i: dr.get_labels('atm', i))
df["col_lbl"] = df['col'].apply(lambda i: dr.get_labels('col', i))

df = df.drop(columns=['an_nais', 'age'], axis=1)

columns_titles = ['Tué', 'Blessé hospitalisé', 'Blessé léger', 'Indemne']

## Relation Gravité - Luminosité

In [ ]:
df_cross = pd.crosstab(df['luminosité'], df['gravité'], normalize='index')
df_cross = df_cross.reindex(columns=columns_titles)
print(df_cross)

sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
chart = df_cross.plot(kind="bar", stacked=True, rot=0, ax=ax, 
                         title='Distribution de la gravité en fonction de la luminosité')
chart.set_xticklabels(chart.get_xticklabels(), rotation=80)
sns.move_legend(ax, "upper right")
